In [177]:
import pandas as pd
df = pd.read_csv('./data/movies_train.csv')
testdf = pd.read_csv('./data/movies_test.csv')

# 전처리

In [178]:
# column 삭제
def delete_col(df) :
    del_list = ['title','dir_prev_bfnum' ,'dir_prev_num','num_staff','num_actor','director'] 
    for i in del_list :
        del df[i]
    return df


# column 생성
def make_col(dataframe, column_name, mapper) :
    new = []
    for i in dataframe[column_name] :
        if i in mapper.keys() :
            new.append(mapper.get(i))
        else :
            new.append(1)
    dataframe['new_' + str(column_name)] = pd.DataFrame(new)
    del dataframe[column_name]
    return dataframe

# 배급사 mapping
def distributor(df, distributor_mapper) :
    return make_col(df, 'distributor', distributor_mapper )
    

# genre mapping
def genre(df) :
    tmp = pd.DataFrame(pd.get_dummies(df['genre'], prefix='genre'))
    df = pd.concat([df, tmp], axis=1)
    del df['genre']
    return df

# release time
def release_time(df) :
    years, months, days = [], [], []
    for line in df['release_time'] :
        a = line.split('-')
        years.append(int(a[0]))
        months.append(int(a[1]))
        days.append(a[2])
        
    new_days = []
    for i in range(len(days)) :
        if int(days[i]) >= 20 :
            new_days.append(3)
        elif int(days[i]) >= 10 :
            new_days.append(2)
        else :
            new_days.append(1)    
            
    df['release_year'] = pd.DataFrame(years)
    df['release_month'] = pd.DataFrame(months)
    df['release_day'] = pd.DataFrame(new_days)

    del df['release_time']
    return df

# time 
def time(df) :
    new_time = []
    for i in df['time'] :
        if i >= 130 :
            new_time.append(3)
        elif i > 70 : 
            new_time.append(2)
        else :
            new_time.append(1)

    df['new_time'] = pd.DataFrame(new_time)
    del df['time']
    return df

# screen_rat
def screen_rat(df) :
    tmp = sorted(df['screening_rat'].unique(), reverse=False)

    mapping = {}
    num = 1
    for i in tmp :
        mapping.update({i : num})
        num += 1

    return make_col(df, 'screening_rat', mapping)
    

In [179]:
# 배급사 mapper는 정해둔다
distributor_mapper = df['distributor'].value_counts()
distributor_mapper.value_counts()

mapping = {}
for i in range(len(distributor_mapper)):

    if distributor_mapper[i] >= 30 :
        mapping.update({ distributor_mapper.index[i] : 4 })

    elif distributor_mapper[i] >= 10 :
        mapping.update({ distributor_mapper.index[i] : 3 })
        
    elif distributor_mapper[i] >= 2 :
        mapping.update({ distributor_mapper.index[i] : 2 })
        
    else :
        mapping.update({ distributor_mapper.index[i] : 1 })


In [180]:
delete_col(df)
distributor(df, mapping)
genre(df)
release_time(df)
time(df)
screen_rat(df)

del df['genre']
df

,num_staff,num_actor,box_off_num,new_distributor,release_year,release_month,release_day,new_time,new_screening_rat
0,91,2,23398,4,2012,11,3,2,4
1,387,3,7072501,3,2015,11,2,3,4
2,343,4,6959083,3,2013,6,1,2,2
3,20,6,217866,4,2012,7,2,2,3
4,251,2,483387,2,2010,11,1,2,2
...,...,...,...,...,...,...,...,...,...
595,510,7,1475091,4,2014,8,2,2,4
596,286,6,1716438,3,2013,3,2,2,2
597,123,4,2475,3,2010,9,3,2,4
598,431,4,2192525,4,2015,5,2,2,2


In [181]:
delete_col(testdf)
distributor(testdf, mapping)
genre(testdf)
release_time(testdf)
time(testdf)
screen_rat(testdf)

del testdf['genre']
testdf

,num_staff,num_actor,new_distributor,release_year,release_month,release_day,new_time,new_screening_rat
0,304,3,2,2010,1,1,2,4
1,275,3,3,2010,1,2,2,1
2,419,7,4,2010,1,3,2,1
3,408,2,3,2010,2,1,2,2
4,380,1,4,2010,2,2,2,2
...,...,...,...,...,...,...,...,...
238,4,4,1,2015,11,2,2,2
239,18,2,3,2015,10,3,2,1
240,224,4,1,2015,10,3,2,4
241,68,7,1,2015,10,3,2,4


# 모델링

In [182]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

In [117]:
# 저장함수
def save_file(prediction, number) :
    sub = pd.read_csv('./data/submission.csv')
    sub['box_off_num'] = prediction
    sub.to_csv('./submission/submission' + str(number) + '.csv', index=False)
    return print('done')

## 사이킷런

In [183]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
lm.fit(train_X, train_Y)

prediction = lm.predict(test_X)

In [119]:
save_file(prediction, 1)
# 1744049.2329963981	

done


In [184]:
save_file(prediction, 4)
# 1509223.1428086378	 , 'num_staff','num_actor'를 추가하였을 때

done


## tensorflow

In [185]:
import tensorflow as tf
import numpy as np

X = tf.placeholder(shape = [None,8], dtype=tf.float32)
Y = tf.placeholder(shape = [None,1], dtype=tf.float32)

W = tf.Variable(tf.random_normal([8,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

H = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(H-Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0000001)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(300000) :
    _, cost_val = sess.run([train, cost],
                          feed_dict = {
                              X : train_X.values,
                              Y : train_Y.values.reshape(-1,1)
                          })
    if step % 30000 == 0 :
        print('cost값은 : {}'.format(cost_val))
    
prediction = sess.run(H, feed_dict = {X : test_X.values, })

cost값은 : 3835362017280.0
cost값은 : 2346045669376.0
cost값은 : 2344372928512.0
cost값은 : 2342798229504.0
cost값은 : 2341311610880.0
cost값은 : 2339906256896.0
cost값은 : 2338578497536.0
cost값은 : 2337319944192.0
cost값은 : 2336127975424.0
cost값은 : 2334994464768.0


In [172]:
save_file(prediction, 2)
# learning_rate = 0.0000001
# 30000번 학습
# 	1857916.1853515022	 

done


In [176]:
save_file(prediction, 3)
# 300000번 학습
# 1844690.7292044302	

done


In [186]:
save_file(prediction, 5)
# num_staff','num_actor'를 추가하였을 때 
# 	1493283.9481655352	

done
